### Setting Up Conda Environment with Python 3.12 and Installing Required Libraries
##### Step 1 - Created a Conda Environment with Python 3.12 and Connected to its Kernel
I created a new Conda environment with Python 3.12 and activated it. After that, I connected to the Python kernel in Jupyter/VSCode using this environment.

Commands to create and activate the environment:
```bash
conda create -n llm_env python=3.12
conda activate llm_env
```

##### Step 2 - Imported Required Libraries
Once the environment was set up and activated, I imported the following libraries for my project:
- `numpy` as `np`: For numerical operations and working with arrays.
- `pandas` as `pd`: For data manipulation and analysis.
- `torch`: PyTorch, for deep learning model building and training.
- `tqdm.notebook`: For displaying progress bars in Jupyter notebooks.
- `sklearn.preprocessing.LabelEncoder`: For encoding categorical labels into numerical format.
- `sklearn.model_selection.train_test_split`: For splitting datasets into training and testing sets.
- `transformers.BertTokenizer`: For tokenizing text data for BERT models.
- `transformers.BertForSequenceClassification`: For using BERT for sequence classification tasks.
- `torch.utils.data.TensorDataset`: For creating datasets from tensors.
- `torch.utils.data.DataLoader`: For batching datasets and loading them efficiently during training.
- `torch.utils.data.RandomSampler` and `torch.utils.data.SequentialSampler`: For random and sequential sampling of datasets.
- `transformers.AdamW` and `transformers.get_linear_schedule_with_warmup`: For the AdamW optimizer and learning rate scheduler used with transformers.
- `sklearn.metrics.f1_score`: For evaluating model performance using the F1 score.
- `random`: For random number generation, typically used in data shuffling.

Commands to install the libraries:
```bash
conda install numpy pandas scikit-learn tqdm pytorch
conda install -c huggingface transformers
```

Once the libraries were installed, I successfully imported them into the notebook and was ready to proceed with building and training the machine learning model.

In [72]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import datetime
import random

In [73]:
num_epochs = 4
learning_rate = 5e-5
num_folds  = 5 
batch_size = 16
# output_model_path = "bert_model_final.pth"
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_model_path = f"model_{timestamp}.pth"

### Set the model to be deterministic

In [74]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

### Data Load and Handling

- **`df`**: The full dataset loaded from the CSV (`dataset.csv`).
- **`labels`**: The list of labels (i.e., 'Artifact Id') extracted from the dataset.
- **`label_counts`**: The count of occurrences of each unique label in the dataset.
- **`filtered_labels`**: The labels that appear at least 5 times in the dataset.
- **`filtered_labels_list`**: A list of labels that appear at least 5 times.
- **`filtered_df`**: The filtered DataFrame containing only the rows with labels that appear at least 5 times.

By running this script, you get a new DataFrame, `filtered_df`, containing only the rows with labels that appear 5 or more times in the original dataset.

**Note**: This is a very simple preprocessing step where we filter out labels with fewer than 5 occurrences. There is no further data cleaning, such as handling missing values or data normalization, applied at this stage.

In [75]:
# df = pd.read_csv('dataset.csv')

# labels = df['Artifact Id']

# label_counts = labels.value_counts()

# filtered_labels = label_counts[label_counts >= 5]

# filtered_labels_list = filtered_labels.index.tolist()

# filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

# print(filtered_df['Artifact Id'].value_counts())

### Testing for Overfitting on a Small Portion of the Data

In this part of the code, we filter the dataset to only include labels that have exactly **16 samples**. By training the model on this small subset, we aim to observe if it achieves **overfitting**, where it memorizes the data rather than generalizing well. This helps us test the model's ability to avoid overfitting on limited data.


In [76]:
# df = pd.read_csv('dataset.csv')

# labels = df['Artifact Id']

# label_counts = labels.value_counts()

# filtered_labels = label_counts[label_counts == 16]

# filtered_labels_list = filtered_labels.index.tolist()

# filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

### Balancing the Dataset

To address class imbalance in the dataset, we performed the following steps:

1. **Random Sampling of 'Command' Class**: We noticed that the **'d3f:Command'** class was underrepresented in the dataset. Therefore, we **randomly sampled 16 instances** from this class to ensure it is adequately represented.

2. **Combining the Data**: After sampling the 'Command' class, we combined it with the filtered dataset to create a **more balanced dataset**.

This process ensures that the model is not biased towards the larger classes and helps it generalize better across all classes.


In [77]:
df = pd.read_csv('dataset.csv')

labels = df['Artifact Id']

label_counts = labels.value_counts()

filtered_labels = label_counts[(label_counts >= 5) & (label_counts <= 200)] # Remove "Command" label

filtered_labels_list = filtered_labels.index.tolist()

filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

command_df = df[df['Artifact Id'] == 'd3f:Command']
sample_size = min(len(command_df), 16)
sampled_command_df = command_df.sample(n=sample_size, random_state=42)
combined_df = pd.concat([filtered_df, sampled_command_df])

combined_df.reset_index(drop=True, inplace=True)

print(combined_df['Artifact Id'].value_counts())

Artifact Id
d3f:Database                     16
d3f:Software                     16
d3f:Command                      16
d3f:HardwareDriver               14
d3f:DisplayServer                11
d3f:OperatingSystem               8
d3f:FileSystem                    7
d3f:BootLoader                    6
d3f:InterprocessCommunication     5
Name: count, dtype: int64


### Divide to Train, Validation, and Test While Keeping the Distribution

In this step, the dataset is split into a training set, a validation set, and a testing set, while maintaining the same label distribution across all sets. This is done using the `train_test_split` function from `sklearn.model_selection`, with the `stratify` parameter set to the labels (`Artifact Id`) to ensure that all splits preserve the same class proportions as the original dataset.

- **Training Set**: Contains 64% of the data.
- **Validation Set**: Contains 16% of the data.
- **Test Set**: Contains 20% of the data.

By using stratified splitting, the label distribution in all sets (training, validation, and test) is consistent, preventing potential bias caused by imbalanced labels in smaller datasets.


In [78]:
# train_df, test_df = train_test_split(filtered_df,
#                                      test_size=0.2,
#                                      stratify=filtered_df['Artifact Id'],
#                                      random_state=42)

# train_df, val_df = train_test_split(train_df,
#                                     test_size=0.2,
#                                     stratify=train_df['Artifact Id'],
#                                     random_state=42)

train_val_df, test_df = train_test_split(filtered_df,
                                         test_size=0.2,
                                         stratify=filtered_df['Artifact Id'],
                                         random_state=42)

In [79]:
# print("Training set label distribution:")
# print(train_df['Artifact Id'].value_counts())

In [80]:
# print("\nValidation set label distribution:")
# print(val_df['Artifact Id'].value_counts())

In [81]:
# print("\nTest set label distribution:")
# print(test_df['Artifact Id'].value_counts())

### Model and Tokenizer Initialization

- **Model**: A pre-trained `BertForSequenceClassification` model is loaded from `bert-base-uncased`, with a classification head set to `num_labels` (number of unique labels in the dataset).

- **Tokenizer**: The `BertTokenizer` for `bert-base-uncased` is used for consistent tokenization.

These components enable fine-tuning of the BERT model for the multiclass classification task.


In [82]:
# model = BertForSequenceClassification.from_pretrained(
#     'bert-base-uncased', num_labels=len(filtered_labels_list))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Data Preparation: Tokenization and Dataloader Creation

- **`tokenize_data`**: Tokenizes input text using the BERT tokenizer, padding and truncating to a max length of 512 tokens, and returns PyTorch tensors.

- **Tokenization**: The training, validation, and test data (`'Example Description'`) are tokenized into input IDs and attention masks.

- **Label Encoding**: Labels (`'Artifact Id'`) are mapped to indices and converted to tensor labels for all datasets.

- **Dataset Creation**: `TensorDataset` objects are created for training, validation, and test data with tokenized inputs and labels.

- **DataLoader**: Three DataLoaders are created:
  - `train_loader`: Random sampling for training.
  - `val_loader` and `test_loader`: Sequential sampling for validation and testing, with a batch size of 16.

These steps prepare the data for efficient use in training, validation, and evaluation.


In [83]:
def tokenize_data(texts, tokenizer):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
# # Tokenize the training, validation, and test data
# train_encodings = tokenize_data(train_df['Example Description'].tolist())
# val_encodings = tokenize_data(val_df['Example Description'].tolist())  # Validation set
# test_encodings = tokenize_data(test_df['Example Description'].tolist())

# # Convert labels to tensors
# train_labels = torch.tensor(train_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())
# val_labels = torch.tensor(val_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())  # Validation set
# test_labels = torch.tensor(test_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())

# train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
# val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)  # Validation set
# test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
# val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)  # Validation set
# test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=16)


### Optimizer and Learning Rate Scheduler

- **Optimizer**: 
  - `AdamW` is used, suitable for fine-tuning transformer models like BERT, with a learning rate of `5e-5`.
  
- **Learning Rate Scheduler**: 
  - A linear scheduler adjusts the learning rate over `total_steps` (batches × epochs), with no warm-up phase (`num_warmup_steps=0`).

These components ensure efficient model optimization during training.


In [84]:
# optimizer = AdamW(model.parameters(), lr=learning_rate)

# total_steps = len(train_loader) * num_epochs

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

### Training Function

This function trains the model for a specified number of epochs:

1. **Forward Pass**: For each batch, the model computes the loss and logits, then makes predictions by selecting the class with the highest logit.
2. **Metrics Calculation**: It tracks the total loss, accuracy, and F1 score across all batches in each epoch. Accuracy is calculated by comparing the predicted labels to the true labels, and the F1 score is computed using the weighted average across all classes.
3. **Optimization**: The model parameters are updated using backpropagation (`loss.backward()`), and the optimizer and learning rate scheduler are updated each step.

At the end of each epoch, the loss, accuracy, and F1 score are printed.



In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [86]:
def train(model, train_loader, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        all_labels = []
        all_preds = []

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            preds = torch.argmax(logits, dim=-1)

            # Store predictions and labels for F1 score calculation
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            # Accuracy calculation
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += labels.size(0)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        # Calculate metrics for the epoch
        accuracy = total_correct / total_samples * 100
        f1 = f1_score(all_labels, all_preds, average="weighted")

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%, F1 Score: {f1:.4f}")

### Training with Validation

The `train_with_validation` function trains the model for multiple epochs and evaluates it on a validation set after each epoch. 

1. **Training**: For each batch, the model computes the loss, updates parameters, and tracks accuracy and F1 score.
2. **Validation**: After each epoch, the model's performance is evaluated on the validation set, and validation accuracy and F1 score are calculated.

Metrics (training and validation accuracy, F1 score) are printed after each epoch.


In [87]:
def train_with_validation(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        all_labels = []
        all_preds = []

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += labels.size(0)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        # Calculate training metrics
        accuracy = total_correct / total_samples * 100
        f1 = f1_score(all_labels, all_preds, average="weighted")
        
        # Validation phase
        model.eval()
        val_correct = 0
        val_samples = 0
        val_all_labels = []
        val_all_preds = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}"):
                input_ids, attention_mask, labels = [item.to(device) for item in batch]

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                preds = torch.argmax(logits, dim=-1)

                val_all_preds.extend(preds.cpu().tolist())
                val_all_labels.extend(labels.cpu().tolist())

                correct = (preds == labels).sum().item()
                val_correct += correct
                val_samples += labels.size(0)

        val_accuracy = val_correct / val_samples * 100
        val_f1 = f1_score(val_all_labels, val_all_preds, average="weighted")

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.6f}, Accuracy: {accuracy:.6f}%, F1 Score: {f1:.6f}")
        print(f"Validation Accuracy: {val_accuracy:.6f}%, Validation F1 Score: {val_f1:.6f}")

    return val_accuracy, val_f1


In [88]:
# train(
#     model=model,
#     train_loader=train_loader,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     device=device,
#     num_epochs=num_epochs
# )

# train_with_validation(
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     device=device,
#     num_epochs=num_epochs
# )

### Model Evaluation

After training, the model is evaluated on the test set to assess its performance on unseen data. During this phase, the model is set to evaluation mode using `model.eval()`, and predictions are made on the test set.

- **Prediction**: For each batch in the test set, the model generates predictions (logits), which are converted to class labels using `torch.argmax()`.
  
- **F1 Score**: The F1 score, calculated using the `f1_score` function from `sklearn.metrics`, is reported to measure the weighted average performance across all classes.
  
- **Accuracy**: The accuracy of the model is calculated by comparing the predicted labels to the true labels and computing the percentage of correct predictions.

The evaluation results help in understanding the model's effectiveness and generalization to the test set.

*Note: This evaluation process does not update the model parameters, ensuring that it is a true test of performance.*


In [89]:
def test(model, test_loader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad(): 
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate F1 score (weighted) for multiclass classification
    f1 = f1_score(true_labels, predictions, average='weighted')

    # Calculate accuracy
    accuracy = np.sum(np.array(predictions) == np.array(true_labels)) / len(true_labels) * 100

    print(f"F1 Score (Weighted): {f1:.4f}")
    print(f"Accuracy: {accuracy:.2f}%")

In [90]:
# test(model, test_loader, device)


### K-Fold Cross Validation with BERT

This code performs **K-Fold Cross Validation** using **StratifiedKFold** for evaluating a BERT-based model on a classification task. For each fold, it:

1. Splits the dataset into training and validation sets.
2. Tokenizes the input text (e.g., "Example Description").
3. Prepares and loads data into `DataLoader` for training and validation.
4. Initializes the BERT model and optimizer, and trains the model for a specified number of epochs.
5. Evaluates the model on the validation set using **F1 score** (weighted) and **accuracy**.
6. Computes average metrics across all folds.

Finally, the trained model is evaluated on a test set, and the final performance metrics are printed.

In [91]:
test_encodings = tokenize_data(test_df['Example Description'].tolist(), tokenizer)
label_mapping = {label: idx for idx, label in enumerate(filtered_labels_list)}
test_labels = torch.tensor([label_mapping.get(x, -1) for x in test_df['Artifact Id']])
test_dataset = TensorDataset(
    test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(
    test_dataset), batch_size=batch_size)

In [92]:
# K-Fold Cross Validation on 80% train/val data
best_model = None
best_val_f1 = 0.0
fold_results = []  # Store F1 score and accuracy for each fold
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(skf.split(train_val_df, train_val_df['Artifact Id'])):
    print(f"\n--- Fold {fold+1}/{num_folds} ---")

    # Split the train/val set into training and validation data for this fold
    train_df = train_val_df.iloc[train_index]
    val_df = train_val_df.iloc[val_index]

    # Tokenize the training and validation data
    train_encodings = tokenize_data(train_df['Example Description'].tolist(), tokenizer)
    val_encodings = tokenize_data(val_df['Example Description'].tolist(), tokenizer)

    # Map labels to indices for training and validation
    train_labels = torch.tensor(train_df['Artifact Id'].map(
        lambda x: filtered_labels_list.index(x)).tolist())
    val_labels = torch.tensor(val_df['Artifact Id'].map(
        lambda x: filtered_labels_list.index(x)).tolist())

    # Create TensorDatasets for training and validation
    train_dataset = TensorDataset(
        train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
    val_dataset = TensorDataset(
        val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

    # Create DataLoaders for training and validation
    train_loader = DataLoader(train_dataset, sampler=RandomSampler(
        train_dataset), batch_size=batch_size)
    val_loader = DataLoader(val_dataset, sampler=SequentialSampler(
        val_dataset), batch_size=batch_size)

    # Initialize model, optimizer, and scheduler
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', num_labels=len(filtered_labels_list))
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )

    # Train the model with validation
    accuracy, f1 = train_with_validation(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        num_epochs=num_epochs
    )
    
    fold_results.append((accuracy, f1))
    if f1 > best_val_f1:
        best_val_f1 = f1
        best_model = model.state_dict()

# # Average results over all folds
# average_f1 = np.mean([result[0] for result in fold_results])
# average_accuracy = np.mean([result[1] for result in fold_results])
# print(f"\n--- K-Fold Results ---")
# print(f"Average F1 Score (Weighted): {average_f1:.4f}")
# print(f"Average Accuracy: {average_accuracy:.2f}%")

# Test the model on the test set
test(model, test_loader, device)

c:\Users\tamar\anaconda3\envs\llm_env\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(



--- Fold 1/5 ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\tamar\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Epoch 1/4:   0%|          | 0/4 [00:00<?, ?it/s]

Validating Epoch 1/4:   0%|          | 0/1 [00:00<?, ?it/s]

...

Training Epoch 4/4:   0%|          | 0/4 [00:00<?, ?it/s]

Validating Epoch 4/4:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 4, Loss: 1.794583, Accuracy: 43.396226%, F1 Score: 0.329977
Validation Accuracy: 30.769231%, Validation F1 Score: 0.164835

--- K-Fold Results ---
Average F1 Score (Weighted): 28.7912
Average Accuracy: 0.18%


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

F1 Score (Weighted): 0.2941
Accuracy: 41.18%


In [93]:
# torch.save(model.state_dict(), output_model_path)
# torch.save({
#     'model': model,  # Save the entire model
#     'optimizer': optimizer,  # Save the entire optimizer object
#     'scheduler': scheduler,  # Save the entire scheduler object
#     'tokenizer': tokenizer,  # Save the tokenizer
#     'label_mapping': filtered_labels_list  # Save label mappings
# }, output_model_path)
# print(f"Model saved to {output_model_path}")

In [94]:
torch.save({
    'model_state_dict': best_model,
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': filtered_labels_list
}, output_model_path)
print(f"Model saved to {output_model_path}")

Model saved to model_2024-12-03_16-37-16.pth


In [95]:
test_df.to_csv('test_data.csv', index=False)
print("DataFrame saved to test_data.csv")

DataFrame saved to test_data.csv
